In [1]:
import pandas as pd
import numpy as np
import warnings
from jenkspy import JenksNaturalBreaks
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv("DS_test_reformulation.csv", sep=";").drop(columns=["Unnamed: 0"])

In [3]:
def str_to_float(value):
    # Replace ',' with '.' and convert to float
    return float(value.replace(',', '.'))

In [4]:
dictionary_to_number = {"mobile": 1, "laptop": 2}
raw_data['device'] = raw_data['device'].map(dictionary_to_number)
raw_data["percentual_price_increment"] = raw_data.percentual_price_increment.apply(str_to_float)
raw_data["profit_per_order"] = raw_data.profit_per_order.apply(str_to_float)
raw_data["profit"] = raw_data.profit.apply(str_to_float)
raw_data['date'] = pd.to_datetime(raw_data['date'])

In [5]:
raw_data

,date,weekday,device,percentual_price_increment,traffic,orders,profit,profit_per_order
0,2022-01-01,6,1,0.150,2225,8,357.50,44.687500
1,2022-01-02,7,1,0.075,2348,15,712.50,47.500000
2,2022-01-03,1,1,-0.025,2473,49,1653.75,33.750000
3,2022-01-04,2,1,-0.050,2320,37,1435.00,38.783784
4,2022-01-05,3,1,-0.025,2762,56,2135.00,38.125000
...,...,...,...,...,...,...,...,...
995,2023-05-11,4,2,-0.100,2526,71,2080.00,29.295775
996,2023-05-12,5,2,0.100,1637,47,2480.00,52.765957
997,2023-05-13,6,2,0.025,2162,20,778.75,38.937500
998,2023-05-14,7,2,-0.200,2833,52,1580.00,30.384615


In [6]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    percentual_price_increment_filter = df["percentual_price_increment"] != 0
    df_first_cleaned = df[percentual_price_increment_filter]
    return df_first_cleaned.copy(deep=True)

raw_data_cleaned = clean_data(raw_data)
raw_data_cleaned

,date,weekday,device,percentual_price_increment,traffic,orders,profit,profit_per_order
0,2022-01-01,6,1,0.150,2225,8,357.50,44.687500
1,2022-01-02,7,1,0.075,2348,15,712.50,47.500000
2,2022-01-03,1,1,-0.025,2473,49,1653.75,33.750000
3,2022-01-04,2,1,-0.050,2320,37,1435.00,38.783784
4,2022-01-05,3,1,-0.025,2762,56,2135.00,38.125000
...,...,...,...,...,...,...,...,...
995,2023-05-11,4,2,-0.100,2526,71,2080.00,29.295775
996,2023-05-12,5,2,0.100,1637,47,2480.00,52.765957
997,2023-05-13,6,2,0.025,2162,20,778.75,38.937500
998,2023-05-14,7,2,-0.200,2833,52,1580.00,30.384615


In [7]:
import statsmodels.api as sm
from matplotlib import pyplot as plt

In [47]:
def get_price_elasticity_mean(group: pd.DataFrame) -> float:
    price_elasticity_grouped_df = group["response"] / group["percentual_price_increment"]
    return price_elasticity_grouped_df.mean()

def get_price_elasticity_slope(group: pd.DataFrame) -> dict:
    X = group[['percentual_price_increment']]
    y = group["response"]
    model = sm.OLS(y, sm.add_constant(X)).fit()
    intercept = model.params[0]
    slope = model.params[1]
    
    return {"slope": slope, "intercept": intercept}

def plotter(group: pd.DataFrame, price_elasticity_slope: dict) -> None:
    fig = plt.figure()
    x = np.linspace(group['percentual_price_increment'].min(), group['percentual_price_increment'].max(), 50)
    intercept = price_elasticity_slope["intercept"]
    slope = price_elasticity_slope["slope"]
    y = intercept + slope * x
    plt.scatter(group['percentual_price_increment'], group['response'], label='Data')
    plt.plot(x, y, color='red', label='Linear Price Response')

    plt.xlabel('Price')
    plt.ylabel('Response')
    plt.title('Linear Price Response Function')

    plt.legend()
    plt.savefig('linear_response.png')
    
def analysis(group):
    group_sorted = group.sort_values(by="date")
    group_sorted["response"] = group_sorted["orders"].pct_change() 
    group_sorted["price_elasticity"] = group_sorted["response"] / group_sorted["percentual_price_increment"] 
    group_sorted.dropna(inplace=True) 
    price_elasticity_slope_dict = get_price_elasticity_slope(group_sorted)
    data = pd.DataFrame([{
        "price_elasticity_mean": get_price_elasticity_mean(group_sorted),
#         "price_elasticity_slope": price_elasticity_slope_dict["slope"],
        "percentual_price_increment_mean": group_sorted.percentual_price_increment.mean(),
        "sum_profit": group_sorted.profit.sum(),
        "traffic": group_sorted.traffic.sum()}])
#     plotter(group_sorted, price_elasticity_slope_dict)
    return data



raw_data_clusterized = raw_data_cleaned.groupby(["weekday", "device"]).apply(analysis)
raw_data_clusterized_sorted = raw_data_clusterized.sort_values(by="price_elasticity_mean", ascending=False).reset_index(drop=True)
raw_data_clusterized_sorted  

,price_elasticity_mean,percentual_price_increment_mean,sum_profit,traffic
0,0.168488,-0.011742,92561.25,148082
1,-0.296991,-0.007031,184147.50,147383
2,-0.337669,-0.020652,111126.25,162118
3,-0.654462,-0.006250,104857.50,148107
4,-0.822942,-0.007463,93026.25,149291
5,-1.160274,-0.009615,187920.00,153774
6,-1.220388,0.007836,110330.00,153702
7,-1.235212,-0.008582,154531.25,150502
8,-1.834472,-0.013060,106145.00,155493
9,-2.570127,-0.009701,151917.50,153464


In [49]:
def jenks_natural_breaks(df: pd.DataFrame, final_grouped_clusters: int) -> dict:
    label_dict = {}
    jnb = JenksNaturalBreaks(final_grouped_clusters)
    jnb.fit(df.price_elasticity_mean)
    for cluster_id, label in enumerate(jnb.labels_):
        label_dict[cluster_id] = label
    
    return label_dict

In [50]:
label_dict = jenks_natural_breaks(raw_data_clusterized_sorted, 3)
label_dict_desc = {0: 'elastic', 1: 'medium', 2: 'inelastic'}
raw_data_clusterized_sorted['cluster'] = [label_dict_desc[label_dict[i]] for i in range(len(raw_data_clusterized_sorted))]
raw_data_clusterized_sorted

,price_elasticity_mean,percentual_price_increment_mean,sum_profit,traffic,cluster
0,0.168488,-0.011742,92561.25,148082,inelastic
1,-0.296991,-0.007031,184147.50,147383,inelastic
2,-0.337669,-0.020652,111126.25,162118,inelastic
3,-0.654462,-0.006250,104857.50,148107,inelastic
4,-0.822942,-0.007463,93026.25,149291,inelastic
5,-1.160274,-0.009615,187920.00,153774,inelastic
6,-1.220388,0.007836,110330.00,153702,inelastic
7,-1.235212,-0.008582,154531.25,150502,inelastic
8,-1.834472,-0.013060,106145.00,155493,inelastic
9,-2.570127,-0.009701,151917.50,153464,inelastic


In [63]:
df_segments = pd.DataFrame(raw_data_clusterized_sorted.groupby("cluster").value_counts())
df_segments

0
cluster   price_elasticity_mean percentual_price_increment_mean sum_profit traffic   
elastic   -15.299643             0.021269                       45296.25   156163   1
inelastic -3.798222              0.008730                       52895.00   145437   1
          -2.570127             -0.009701                       151917.50  153464   1
          -1.834472             -0.013060                       106145.00  155493   1
          -1.235212             -0.008582                       154531.25  150502   1
          -1.220388              0.007836                       110330.00  153702   1
          -1.160274             -0.009615                       187920.00  153774   1
          -0.822942             -0.007463                       93026.25   149291   1
          -0.654462             -0.006250                       104857.50  148107   1
          -0.337669             -0.020652                       111126.25  162118   1
          -0.296991             -0.007031                       184147.50  147383   1
           0.168488             -0.011742                       92561.25   148082   1
medium    -8.302405              0.016304                       47718.75   157962   1
          -7.085678              0.018561                       54552.50   151852   1

In [57]:
def update_percentual_price_increment_mean(df: pd.DataFrame) -> pd.DataFrame:
    max_profit = df['sum_profit'].max()
    value = df.loc[df['sum_profit'] == max_profit, 'percentual_price_increment_mean'].values[0]
    df['percentual_price_increment_mean'] = value
    return df

In [62]:
segments = ['elastic', 'inelastic', 'medium']
raw_data_max_profit = raw_data_clusterized_sorted.groupby('cluster').apply(update_percentual_price_increment_mean)
raw_data_max_profit

,price_elasticity_mean,percentual_price_increment_mean,sum_profit,traffic,cluster
0,0.168488,-0.009615,92561.25,148082,inelastic
1,-0.296991,-0.009615,184147.50,147383,inelastic
2,-0.337669,-0.009615,111126.25,162118,inelastic
3,-0.654462,-0.009615,104857.50,148107,inelastic
4,-0.822942,-0.009615,93026.25,149291,inelastic
5,-1.160274,-0.009615,187920.00,153774,inelastic
6,-1.220388,-0.009615,110330.00,153702,inelastic
7,-1.235212,-0.009615,154531.25,150502,inelastic
8,-1.834472,-0.009615,106145.00,155493,inelastic
9,-2.570127,-0.009615,151917.50,153464,inelastic
